# Surprise homework

In [17]:
import io 
from collections import defaultdict

import pandas as pd
from surprise import Dataset
from surprise import SVD
from surprise import NormalPredictor
from surprise import KNNWithMeans
from surprise import accuracy

from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split

import os

### Functions

In [2]:
# используйте полезные функции из FAQ
def get_top_n(predictions, n=10):
    """Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    """

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

def precision_recall_at_k(predictions, k=10, threshold=3.5):
    """Return precision and recall at k metrics for each user"""

    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():

        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        # Number of relevant items
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)

        # Number of recommended items in top k
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold))
                              for (est, true_r) in user_ratings[:k])

        # Precision@K: Proportion of recommended items that are relevant
        # When n_rec_k is 0, Precision is undefined. We here set it to 0.

        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0

        # Recall@K: Proportion of relevant items that are recommended
        # When n_rel is 0, Recall is undefined. We here set it to 0.

        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 0

    return precisions, recalls

### Load data

In [3]:
data = Dataset.load_builtin('ml-100k')
data

### Define algorithms

In [4]:
# внимательно изучите документацию по метрикам и алгоритмам
normalAlg = NormalPredictor()

sim_options = {'name': 'cosine'}
cosAlg = KNNWithMeans(sim_options=sim_options)

sim_options = {'name': 'pearson_baseline'}
pearsonAlg = KNNWithMeans(sim_options=sim_options)

sim_options = {'name': 'msd'}
msdAlg = KNNWithMeans(sim_options=sim_options)

svdAlg = SVD()

### Select algorithm

In [5]:
cross_validate(normalAlg, data, measures=['RMSE'], cv=5, verbose=True)

Evaluating RMSE of algorithm NormalPredictor on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.5124  1.5155  1.5172  1.5126  1.5188  1.5153  0.0025  
Fit time          0.22    0.26    0.24    0.24    0.23    0.24    0.01    
Test time         0.29    0.25    0.27    0.30    0.31    0.28    0.02    


{'test_rmse': array([1.51235751, 1.51549447, 1.51716755, 1.51258015, 1.51876918]),
 'fit_time': (0.21882915496826172,
  0.25604844093322754,
  0.24122166633605957,
  0.2381887435913086,
  0.2344350814819336),
 'test_time': (0.29325342178344727,
  0.2480623722076416,
  0.2720677852630615,
  0.29696202278137207,
  0.30916500091552734)}

In [6]:
cross_validate(cosAlg, data, measures=['RMSE'], cv=5, verbose=True)

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Evaluating RMSE of algorithm KNNWithMeans on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9595  0.9560  0.9488  0.9572  0.9540  0.9551  0.0036  
Fit time          3.16    3.31    3.20    2.68    2.90    3.05    0.23    
Test time         6.46    6.27    6.32    5.78    6.22    6.21    0.23    


{'test_rmse': array([0.95954247, 0.95597974, 0.94881104, 0.957194  , 0.95401429]),
 'fit_time': (3.1587071418762207,
  3.3094677925109863,
  3.2039668560028076,
  2.6840953826904297,
  2.8987929821014404),
 'test_time': (6.461134672164917,
  6.2651896476745605,
  6.321328401565552,
  5.776730298995972,
  6.222026348114014)}

In [7]:
cross_validate(pearsonAlg, data, measures=['RMSE'], cv=5, verbose=True)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Evaluating RMSE of algorithm KNNWithMeans on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9449  0.9330  0.9323  0.9433  0.9357  0.9378  0.0053  
Fit time          3.47    3.46    3.72    3.78    3.52    3.59    0.13    
Test time         6.27    6.83    6.35    6.31    6.36    6.42    0.21    


{'test_rmse': array([0.94490217, 0.93301222, 0.93230739, 0.94332069, 0.93567794]),
 'fit_time': (3.471088409423828,
  3.458237886428833,
  3.7203166484832764,
  3.7774109840393066,
  3.5220837593078613),
 'test_time': (6.2685887813568115,
  6.83304500579834,
  6.3461363315582275,
  6.310084581375122,
  6.3593525886535645)}

In [12]:
cross_validate(msdAlg, data, measures=['RMSE'], cv=5, verbose=True)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE of algorithm KNNWithMeans on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9420  0.9489  0.9527  0.9526  0.9562  0.9505  0.0048  
Fit time          1.07    0.93    0.93    0.87    1.20    1.00    0.12    
Test time         6.91    7.00    6.43    7.44    7.18    6.99    0.34    


{'test_rmse': array([0.94196486, 0.94887555, 0.95267233, 0.95256295, 0.95618462]),
 'fit_time': (1.073423147201538,
  0.9304840564727783,
  0.928041934967041,
  0.8722121715545654,
  1.2002699375152588),
 'test_time': (6.913332223892212,
  7.003416299819946,
  6.4297497272491455,
  7.441978693008423,
  7.183050870895386)}

In [13]:
cross_validate(svdAlg, data, measures=['RMSE'], cv=5, verbose=True)

Evaluating RMSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9327  0.9395  0.9351  0.9332  0.9388  0.9359  0.0028  
Fit time          7.45    8.15    7.41    7.40    7.64    7.61    0.28    
Test time         0.25    0.34    0.22    0.22    0.41    0.29    0.07    


{'test_rmse': array([0.93273643, 0.93954703, 0.93512531, 0.93320649, 0.93884204]),
 'fit_time': (7.454216480255127,
  8.15038800239563,
  7.4096999168396,
  7.399476766586304,
  7.6424477100372314),
 'test_time': (0.24917197227478027,
  0.3412134647369385,
  0.21881914138793945,
  0.22405719757080078,
  0.4081003665924072)}

### Calculate precision@k and recall@k

In [16]:
trainset, testset = train_test_split(data, test_size=.25)
svdAlg.fit(trainset)
predictions = svdAlg.test(testset)

precisions, recalls = precision_recall_at_k(predictions, k=5, threshold=3.52)
print(pd.Series(precisions).mean(), pd.Series(recalls).mean())

0.7454883227176221 0.37051799967309446


### Predict

In [24]:
# обратите внимание на функцию build_anti_testset
variant = '20'
userPredict = get_top_n(predictions, 5)[variant]

In [25]:
itemPath = os.path.expanduser('~/.surprise_data/ml-100k/ml-100k/u.item')
itemDf = pd.read_csv(itemPath, sep='|',encoding='ISO-8859-1', header = None, index_col = 0)

print("User " + variant)
for item in userPredict:
    film = itemDf.loc[int(item[0])][1]
    date = itemDf.loc[int(item[0])][2]
    print(item[0] + " ('" + film + "', '" + date + "') " + str(round(item[1], 3)))

User 1
200 ('Shining, The (1980)', '01-Jan-1980') 4.524
114 ('Wallace & Gromit: The Best of Aardman Animation (1996)', '05-Apr-1996') 4.494
191 ('Amadeus (1984)', '01-Jan-1984') 4.472
199 ('Bridge on the River Kwai, The (1957)', '01-Jan-1957') 4.45
136 ('Mr. Smith Goes to Washington (1939)', '01-Jan-1939') 4.443
